In [1]:
# Import Dependancies
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

# Function to clean suburb names by converting to lowercase 
def clean_suburb_name(suburb):
    suburb = suburb.lower()  # Convert to lowercase
    return suburb

In [2]:
# Import suburbs
LANDING_DATA_DIR = "../../data/landing"
RAW_DATA_DIR = "../../data/raw"
CURATED_DATA_DIR = "../../data/curated"
vic_suburbs = gpd.read_file(f"{LANDING_DATA_DIR}/GDA2020/vic_localities.shp")
vic_suburbs = vic_suburbs.drop_duplicates(subset='LOC_NAME', keep='first')
vic_suburbs['suburb'] = vic_suburbs['LOC_NAME'].apply(clean_suburb_name)

# Import sa2 Mapping
vic_sa2s = pd.read_excel(f"{LANDING_DATA_DIR}/sa2_mapping.xlsx")
vic_sa2s['sa2_suburb'] = vic_sa2s['SA2_NAME_2016'].apply(clean_suburb_name)
vic_sa2s = vic_sa2s[['sa2_suburb','SA2_MAINCODE_2016' ]]

# Join to vic suburbs on suburb name to get sa2 codes: SA2_MAINCODE_2016 
aggregate_df = vic_suburbs[['suburb']].merge(vic_sa2s, left_on='suburb', 
                                             right_on='sa2_suburb', how='left')
aggregate_df = aggregate_df[['suburb', 'SA2_MAINCODE_2016','sa2_suburb' ]]

# Find matching sa2
matched_sa2_list = []

for suburb in vic_suburbs['suburb']:
    pattern = rf'\b{suburb}\b'  # Create a regex pattern with word boundaries
    matching_sa2s = vic_sa2s[vic_sa2s['sa2_suburb']\
                             .str.contains(pattern, case=False, regex=True)]
    matching_sa2s['matched_suburb'] = suburb  
    matched_sa2_list.append(matching_sa2s)

# Combine all matched SA2s into a single DataFrame
matched_sa2_df = pd.concat(matched_sa2_list, ignore_index=True)

# Group by vic suburbs and take max sa2code is there are multiple 
suburb_sa2_df = matched_sa2_df.groupby('matched_suburb')\
    ['SA2_MAINCODE_2016'].max().reset_index()

# Rename columns for the final DataFrame
suburb_sa2_df.columns = ['suburb', 'sa2_code']


C:\Users\DELL 7590\AppData\Local\Temp\ipykernel_17304\501460499.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_sa2s['matched_suburb'] = suburb
C:\Users\DELL 7590\AppData\Local\Temp\ipykernel_17304\501460499.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_sa2s['matched_suburb'] = suburb
C:\Users\DELL 7590\AppData\Local\Temp\ipykernel_17304\501460499.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [3]:
# Join to income to get median income 

# Import Income Data
income_path = f'{LANDING_DATA_DIR}/income_data'
income_df = pd.read_excel(income_path,sheet_name = 'Table 2.4', skiprows=5)
income_df.reset_index(drop=True, inplace=True)

# Merge with suburb_sa2_df
aggregate_df = suburb_sa2_df\
    .merge(income_df, left_on='sa2_code', right_on='Unnamed: 0', how='left')
aggregate_df = aggregate_df[['suburb', 'sa2_code', 'Median', 'Mean']]
aggregate_df.columns = \
    ['suburb', 'sa2_code', 'median_annual_income', 'mean_annual_income']
print(aggregate_df)

           suburb   sa2_code median_annual_income mean_annual_income
0      abbotsford  206071139                67457              81592
1      aberfeldie  206031114                62800              85058
2           agnes  402051055                52383              57295
3    airport west  210011226                60083              67698
4     albert park  404011094                52520              56329
..            ...        ...                  ...                ...
582     yarrambat  209031213                59405              78375
583    yarraville  213031352                71512              85989
584    yarrawonga  216021414                43654              52071
585           yea  204011062                40162              50478
586        yuroke  210051445                52921              57653

[587 rows x 4 columns]


In [4]:
# Import Real Estate Data 
import re
# Define a function to extract the suburb
def extract_suburb(address):
    # Define a regex pattern to match suburb names
    pattern = r'(\w+)\s+(?:VIC|NSW|QLD|WA|SA|TAS|ACT)\s+\d+'
    
    # Search for the pattern in the address
    match = re.search(pattern, address)
    
    # If a match is found, return the matched suburb
    if match:
        return match.group(1)
    else:
        return None


# Get median monthly rent 

real_estate_path = f'{RAW_DATA_DIR}/raw_real_estate_data.csv'
real_estate_df = pd.read_csv(real_estate_path)
real_estate_df['suburb'] = real_estate_df['address'].apply(extract_suburb)
real_estate_df['suburb'] = real_estate_df['suburb'].apply(clean_suburb_name)

# Display the resulting DataFrame
print(real_estate_df.head(20))

                                         listing_link  \
0   https://www.domain.com.au/904-265-exhibition-s...   
1   https://www.domain.com.au/210-422-collins-stre...   
2   https://www.domain.com.au/1902-200-spencer-str...   
3   https://www.domain.com.au/312b-399-bourke-stre...   
4   https://www.domain.com.au/3313-228-la-trobe-st...   
5   https://www.domain.com.au/301-499-st-kilda-roa...   
6   https://www.domain.com.au/2302-17-spring-stree...   
7   https://www.domain.com.au/3201-318-queen-stree...   
8   https://www.domain.com.au/1-113-brook-street-m...   
9   https://www.domain.com.au/403-17-singers-lane-...   
10  https://www.domain.com.au/1413-199-william-str...   
11  https://www.domain.com.au/2408-5-sutherland-st...   
12  https://www.domain.com.au/11-187-collins-stree...   
13  https://www.domain.com.au/1212-555-flinders-st...   
14  https://www.domain.com.au/5-650-elizabeth-stre...   
15  https://www.domain.com.au/32-18-queen-street-m...   
16  https://www.domain.com.au/3

In [5]:
# Define a function to calculate the price per month
def calculate_price_per_month(row):
    if not pd.notna(row['price_per_month']):
        if pd.notna(row['price_per_year']):
            return row['price_per_year'] / 12
        elif pd.notna(row['price_per_week']):
            return row['price_per_week'] *31/ 7 
    return row['price_per_month']

# Apply the function to create a new column 'calculated_price_per_month'
real_estate_df['calculated_price_per_month'] = \
    real_estate_df.apply(calculate_price_per_month, axis=1)

# Display the resulting DataFrame
print(real_estate_df)

                                            listing_link  \
0      https://www.domain.com.au/904-265-exhibition-s...   
1      https://www.domain.com.au/210-422-collins-stre...   
2      https://www.domain.com.au/1902-200-spencer-str...   
3      https://www.domain.com.au/312b-399-bourke-stre...   
4      https://www.domain.com.au/3313-228-la-trobe-st...   
...                                                  ...   
12222  https://www.domain.com.au/10-beacon-crt-inverl...   
12223  https://www.domain.com.au/7-headland-way-inver...   
12224  https://www.domain.com.au/10-blue-wren-way-inv...   
12225  https://www.domain.com.au/18-golf-street-inver...   
12226  https://www.domain.com.au/2-37-a-beckett-stree...   

                                            address      cost_text  \
0      904/265 Exhibition Street Melbourne VIC 3000  $850 Per Week   
1         210/422 Collins Street Melbourne VIC 3000  $475 Per Week   
2        1902/200 Spencer Street Melbourne VIC 3000        $630 pw   

In [6]:
# Group by 'Suburb' and calculate median and mean price per month
suburb_rent_df = real_estate_df.groupby('suburb')['calculated_price_per_month']\
    .agg(['median', 'mean']).reset_index()

# Rename the columns
suburb_rent_df\
    .rename(columns={'median': 'median_price_per_month', 
                     'mean': 'mean_price_per_month'}, inplace=True)

# Display the resulting DataFrame
print(suburb_rent_df)

         suburb  median_price_per_month  mean_price_per_month
0          3004             1815.714286           1815.714286
1    abbotsford             2812.142857           3027.481481
2    aberfeldie             2701.428571           2641.171429
3       aintree             2103.571429           2136.785714
4        albans             1948.571429           1951.925714
..          ...                     ...                   ...
501   yarrambat             5314.285714           5314.285714
502  yarraville             2778.928571           2819.147321
503  yarrawonga             2214.285714           2116.596639
504         yea             1860.000000           1705.000000
505      yinnar             2103.571429           2094.081633

[506 rows x 3 columns]


In [7]:
# Merge the two DataFrames based on the 'Suburb' column
merged_df = aggregate_df.merge(suburb_rent_df, on='suburb', how='left')

# Convert 'Median_Price_Per_Month' and 'Median_Annual_Income' to numeric, 
# setting errors='coerce' to handle non-numeric values
merged_df['median_price_per_month'] = \
    pd.to_numeric(merged_df['median_price_per_month'], errors='coerce')
merged_df['median_annual_income'] = \
    pd.to_numeric(merged_df['median_annual_income'], errors='coerce')

# Calculate affordability metric (income as a percentage of rental prices)
merged_df['affordability'] = \
    (merged_df['median_annual_income'] / 
     (merged_df['median_price_per_month'] * 12)) * 100


# Sort the DataFrame by 'Affordability' column in descending order
sorted_df = merged_df.sort_values(by='affordability', ascending=False)

# Print the top 20 rows
print(sorted_df.head(20))

# Take only relevant columns
sorted_df = sorted_df[['suburb', 'median_annual_income', \
                       'median_price_per_month', 'affordability']]

# Save to curated data folder 
filename = "affordability_ranked.csv"
sorted_df.to_csv(f"{CURATED_DATA_DIR}/{filename}", index=False)

          suburb   sa2_code  median_annual_income mean_annual_income  \
293      kyabram  216011407               44447.0              48712   
496      stawell  215011392               46736.0              52591   
392     numurkah  216021413               44256.0              48472   
32   bannockburn  203011034               57208.0              66336   
372   myrtleford  204031071               43534.0              51543   
359     mortlake  120011383               62464.0              88933   
383        nhill  215011390               49436.0              63556   
276       kerang  215031402               42710.0              48442   
55     blackwood  403031067               59071.0              70897   
71    brookfield  304021087               64899.0             108003   
234     hamilton  507011154               51578.0              59881   
95    camperdown  217031471               47411.0              54546   
6         albion  305031119               61259.0              7

In [8]:
from sklearn.preprocessing import MinMaxScaler

# Calculate affordability metric (income as a percentage of rental prices)
merged_df['affordability'] = \
    ((merged_df['median_annual_income'] / 12) \
     / merged_df['median_price_per_month']) * 100

# Initialize a MinMaxScaler to scale affordability scores between 0 and 100
scaler = MinMaxScaler(feature_range=(0, 100))

# Reshape the 'affordability' column for scaling
affordability_values = merged_df['affordability'].values.reshape(-1, 1)

# Fit and transform the affordability values
scaled_affordability = scaler.fit_transform(affordability_values)

# Assign the scaled values back to the DataFrame
merged_df['affordability'] = scaled_affordability

# Sort the DataFrame by 'Affordability' column in ascending order
sorted_df = merged_df.sort_values(by='affordability', ascending=False)

# Print the top 20 rows
print(sorted_df.head(20))

# Take only relevant columns
sorted_df = sorted_df[['suburb', 'median_annual_income', 
                       'median_price_per_month', 'affordability']]

# Save to curated data folder 
filename = "affordability_ranked.csv"
sorted_df.to_csv(f"{CURATED_DATA_DIR}/{filename}", index=False)

          suburb   sa2_code  median_annual_income mean_annual_income  \
293      kyabram  216011407               44447.0              48712   
496      stawell  215011392               46736.0              52591   
392     numurkah  216021413               44256.0              48472   
32   bannockburn  203011034               57208.0              66336   
372   myrtleford  204031071               43534.0              51543   
359     mortlake  120011383               62464.0              88933   
383        nhill  215011390               49436.0              63556   
276       kerang  215031402               42710.0              48442   
55     blackwood  403031067               59071.0              70897   
71    brookfield  304021087               64899.0             108003   
234     hamilton  507011154               51578.0              59881   
95    camperdown  217031471               47411.0              54546   
6         albion  305031119               61259.0              7